In [1]:
import h5_helpers as h5
import dummydata
from IPython.display import display

## Define Run Parameters and Metadata

In [2]:

filename = 'test.hdf5'
# metadata = {
#     'timestamp'         : False,
#     'element_index'     : False,
#     'hidden'            : False
# }

## Save Dummy Data for all elements into the HDF5 file


In [3]:
print("Generating Dummy h5 file with the following settings")
run_settings = dummydata.defaults
display(run_settings)
dummydata.fill_hdf(run_settings)

Generating Dummy h5 file with the following settings


{'filename': 'test.hdf5',
 'fluid_list': ['water', 'beer1', 'beer2'],
 'wavelength_range': [400, 420, 0.5],
 'instrument': {'Name': 'HAN24',
  'Element_rows': 4,
  'Element_cols': 4,
  'Light Source': 'Stellarnet LED White',
  'Spectrometer': 'Stellarnet BlueWave VIS-25'},
 'elements': 'all',
 'repeats': 3,
 'measuredOn': '2021_10_21'}

## Import from .txt or .csv files

### Input File

Requires a text file with at least 2 columns.
(They don't need to be named in the file)

[wavelength]	[transmission data 1]	...[transmission data n (optional)]

The default is for tab separation, but can be specified with:

separator = '\t'

### Regex
A regex must be provided to identify these metadata fields from the filename:
* sensor
* element_index
* fluid

Optional, will be preserved in the metadata:
* rotation


In [4]:
#Folder containing text/csv files to import
import_dir = "/Users/calum/git/Glasgow/sampleData/Beer x Bitter"
import_dir = "/Users/calum/git/Glasgow/sampleData/combined"

# Construct a regex to extract metadata from the filename
# example 'HAN24_Sensor9_BeerBitter3_Rotation2.txt'
regex = '(?P<sensor>.+)_Sensor(?P<element_index>.+)_(?P<fluid>.+)_Rotation(?P<rotation>.+).txt'

h5.import_dir_to_hdf(import_dir, regex, filename)

importing HAN24_Sensor10_BeerBitter1_Rotation1.txt to HAN24/_2021_10_21/BeerBitter1/_10_rep01
importing HAN24_Sensor10_BeerBitter1_Rotation1.txt to HAN24/_2021_10_21/BeerBitter1/_10_rep02
importing HAN24_Sensor10_BeerBitter1_Rotation1.txt to HAN24/_2021_10_21/BeerBitter1/_10_rep03
importing HAN24_Sensor10_BeerBitter1_Rotation2.txt to HAN24/_2021_10_21/BeerBitter1/_10_rep04
importing HAN24_Sensor10_BeerBitter1_Rotation2.txt to HAN24/_2021_10_21/BeerBitter1/_10_rep05
importing HAN24_Sensor10_BeerBitter1_Rotation2.txt to HAN24/_2021_10_21/BeerBitter1/_10_rep06
importing HAN24_Sensor10_BeerBitter2_Rotation1.txt to HAN24/_2021_10_21/BeerBitter2/_10_rep01
importing HAN24_Sensor10_BeerBitter2_Rotation1.txt to HAN24/_2021_10_21/BeerBitter2/_10_rep02
importing HAN24_Sensor10_BeerBitter2_Rotation1.txt to HAN24/_2021_10_21/BeerBitter2/_10_rep03
importing HAN24_Sensor10_BeerBitter2_Rotation2.txt to HAN24/_2021_10_21/BeerBitter2/_10_rep04
importing HAN24_Sensor10_BeerBitter2_Rotation2.txt to HAN24/

## Inspect an HDF5 file by metadata

In [5]:
h5.inspect('test.hdf5')

Found 945 measurements out of 1350 expected permutations, including:
10 fluids ['Beer', 'BeerBitter1', 'BeerBitter2', 'BeerBitter3', 'BeerCorny1', 'BeerCorny2', 'BeerCorny3', 'BeerMusty1', 'BeerMusty2', 'BeerMusty3']
15 elements ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15']
9 repeats ['01', '02', '03', '04', '05', '06', '07', '08', '09']
0 dates []
1 Data lengths [4095]


## Example of searching by metadata

In [6]:
measurements = h5.measurement_nodes(filename, '/')
A00 = h5.filter_by_metadata(filename, 'element_index', "A00", measurements)
A00_water = h5.filter_by_metadata(filename, 'fluid', "water", A00)

print(A00_water)


[]


## Export Table

In [7]:
measurements = h5.measurement_nodes(filename, '/')
export = h5.export_dataframes(filename, 'export.csv', measurements)
display(export)

Sensor 01                                                  \
wavelength         457.131 457.204 457.277 457.349 457.422 457.495 457.567   
Beer_rep01           81.35   79.39   79.18   79.77   80.53   80.52   81.27   
Beer_rep02           81.35   79.52   78.98   79.76   80.45   80.69   81.30   
Beer_rep03           81.27   79.24   78.88   79.43   80.81   80.99   81.44   
Beer_rep04           82.28   80.34   80.45   80.67   80.49   80.59   80.83   
Beer_rep05           82.30   80.79   80.63   80.56   80.38   80.81   80.90   
...                    ...     ...     ...     ...     ...     ...     ...   
BeerMusty3_rep02     97.01   86.45   81.31   79.26   77.92   77.24   77.23   
BeerMusty3_rep03     97.04   86.62   81.40   79.35   77.99   77.40   77.39   
BeerMusty3_rep04     97.08   87.00   81.77   79.64   78.06   77.39   77.27   
BeerMusty3_rep05     97.22   87.66   82.31   80.15   78.75   77.99   77.87   
BeerMusty3_rep06     96.84   85.74   80.53   78.48   77.05   76.68   76.74   

                                          ... Sensor 15                  \
wavelength       457.640 457.712 457.785  ...   730.219 730.282 730.345   
Beer_rep01         82.23   83.28   84.03  ...    100.92  103.04  100.94   
Beer_rep02         81.84   83.34   84.29  ...    101.83  104.27  101.78   
Beer_rep03         81.75   83.12   84.20  ...    100.92  104.08  104.12   
Beer_rep04         81.11   81.17   81.11  ...     76.82   74.31   75.80   
Beer_rep05         80.92   81.54   81.85  ...     75.30   75.31   76.95   
...                  ...     ...     ...  ...       ...     ...     ...   
BeerMusty3_rep02   77.31   77.29   77.40  ...     62.70   64.31   64.48   
BeerMusty3_rep03   77.45   77.24   77.40  ...     64.36   64.56   63.26   
BeerMusty3_rep04   77.29   77.24   77.31  ...     64.02   64.34   63.75   
BeerMusty3_rep05   77.76   77.64   77.73  ...     64.14   63.91   63.90   
BeerMusty3_rep06   76.88   76.89   77.21  ...     63.38   63.79   64.00   

                                                                          
wavelength       730.408 730.471 730.534 730.598 730.661 730.724 730.787  
Beer_rep01         99.09  102.61  101.41   99.15   97.89  101.12  102.29  
Beer_rep02         99.45  103.26  102.26  100.28   99.08  102.98  103.53  
Beer_rep03        102.37  102.14  103.02  102.91  102.39  102.33  102.86  
Beer_rep04         77.68   77.44   74.39   74.74   76.61   76.16   74.61  
Beer_rep05         75.51   75.35   75.14   76.73   75.08   75.78   75.55  
...                  ...     ...     ...     ...     ...     ...     ...  
BeerMusty3_rep02   63.66   62.39   62.84   63.09   62.95   61.98   62.28  
BeerMusty3_rep03   63.45   63.82   62.66   62.37   62.65   62.90   61.90  
BeerMusty3_rep04   63.51   62.97   62.48   62.40   63.01   62.75   62.13  
BeerMusty3_rep05   64.27   63.43   62.42   62.64   63.61   63.05   61.84  
BeerMusty3_rep06   63.02   63.18   62.78   63.45   62.77   62.72   62.37  

[63 rows x 61425 columns]